In [1]:
import os
import g_requests
import parsers
os.getcwd()
import pandas as pd 

Documentation can be found here - https://sebporras.github.io/GRASP_API/



# Performing joint reconstructions.

Step 1) Submit the job to the server

In [2]:
g_requests.JointReconstruction(aln="../test_data/big_test_data/GRASPTutorial_Final.aln",
                                      nwk="../test_data/big_test_data/GRASPTutorial_Final.nwk")

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



'{"Message":"Queued","Job":31}\n'

Step 2) Find out where your job is in the queue

In [3]:
g_requests.PlaceInQueue(27)

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



'{"Job":27,"Place":0}\n'

Step 3) Retrieve your job which will have the POG graphs 

In [4]:
graphs = g_requests.JobOutput(27)


Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



Step 4 - Optional) Request POGs for extant sequences 

In [5]:
extant_tree = g_requests.ExtantPOGTree(aln="../test_data/big_test_data/GRASPTutorial_Final.aln",
nwk="../test_data/big_test_data/GRASPTutorial_Final.nwk")


Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



Step 5 - Option 1) Build a POG tree from extant and ancesor POGs using both of the server outputs 

- The advantage of doing it this way is that the POGTree object will contain sequence information on the BranchPoints for ancestors AND extants 

In [6]:
tree = parsers.POGTreeFromJSON(extant_tree, graphs)

tree.branchpoints["XP_012687241.1"].seq

'PPGPTPWPLVGNFGGFLVPSFISRRFGRNREEYAQRTSNPVSPQVGLMELSKIYGNIYSIFVGSQLMVLLTGYEVVRDALSNHAEVFSDRPDVPIITIMTKRKGIVFAPYGPVWRKQRKFCHSTLRSFGLGKLSLEPCILEGVSIVKSELLSLSNESGEMGVDPTPLISNAVSNVISSISLGQRFHHQDQEFRTLLDLMARGLEISVNSHAVLINIFPWLYHMPCGVFKELRQVEGDITAFLKRIIARHRATLDPENPRDLIDMYLVEMLAQQTAAEPTENTFSEDYLFYIIGDLFIAGTDTTTNSMLWILLYMCLYPDIQEKVQEEIDAVIGRDKVPSLTDKGCLSFTEATIMEVQRMTVVVPLAIPHMASKTTVFRGYTIPKGTVIIPNLWSVHRDPSVWDSPDDFNPSRFLDEQGKLLRKDYFIPFGIGRRVCMGEQLAKMELFLMFTNLMQAFTFRFPKGKPPPPMHGRFGLTLAPCPYTVCVTPR'

Step 5 - Option 2) Build a POGTree from the ancestor POG and from a nwk file string 

- ONLY ancestors will have sequence information based on the most likely symbol at each position in the sequence

In [3]:
tree = parsers.POGTreeFromJSON(nwk="../test_data/big_test_data/GRASPTutorial_Final.nwk", POG_graphs=graphs)

# Learning distributions from data 

The following instructions demonstrate how to learn a probability distribution from data. I need to add option to change some of the parameters as currently just runs on default settings. 

Step 1 - Optional) The data below is a demonstration of how data should be formated in a CSV.

- Currently only implemented to accept 1 data point per sequence

In [4]:
df = pd.DataFrame()

headers = ["A5ILB0","P08144","P29957","H2N0D4","T1WDH2","T1WE96","H9B4I9","A0A060DAC6","Q47R94","Q5UZY3","D8J7H2","Q4A3E0","L8B068","Q2QC88","O93647","Q4J9M2","Q97YY0","O33476","D0KTV8","P96107","P06279","P04746","P00690","O08452","D4P4Y7","Q1KLC8","Q8A1G3","P17654","B8Y698","P00692","B8Y1H0","B1VK33","B5ARZ9","P20845","B6RB08","P06278","P04063","P00693","Q8LJQ6","Q2KJQ1","Q6WUB6","A8VWC5","A0SEG1","P04745","Q75UG5"]
data = [8.5,7.35,7.35,7,9,9,None,8.5,7.35,None,None,None,7,5.75,5.25,3.25,None,6.5,None,None,None,None,7.35,5.75,5.5,7,None,None,8,6,None,7,6.5,None,6.5,6.466666666666666,None,None,None,None,9.5,None,None,6.9,7]

df["Headers"] = headers
df["Data"] = data

df.to_csv('train_data.csv', index=False)


Step 2) Send request to the server

In [3]:


g_requests.TrainFromData(nwk="./3_2_1_1_filt.nwk", states=["A", "B"], data="train_data.csv")

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



'{"Message":"Queued","Job":32}\n'

Step 3) Check the status of your job 

In [4]:
g_requests.PlaceInQueue(32)

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



'{"Job":32,"Place":0}\n'

Step 4) Retrieve your job and save the output

In [2]:
out = g_requests.JobOutput(32)

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



In [4]:
out

{'Job': 32,
 'Result': {'Distrib': {'Condition': [['A'], ['B']],
   'Pr': [[6.578658560392298, 0.500334272187614],
    [8.632049675842907, 0.500334272187614]],
   'Index': [0, 1],
   'Domain': 'dat.Continuous@56b7d0f2'}}}

Step 5) This distribution can be further refined with requestInferFromData

In [5]:
infer = g_requests.InferFromData(nwk="./3_2_1_1_filt.nwk", 
states=["A", "B"], data="train_data.csv", distrib=out)

{"Command": "Infer", "Auth": "Guest", "Params": {"States": ["A", "B"], "Tree": {"Parents": [-1, 0, 0, 0, 3, 3, 5, 6, 6, 8, 9, 9, 8, 12, 12, 14, 15, 16, 17, 17, 16, 20, 20, 22, 22, 24, 24, 26, 26, 15, 29, 30, 30, 32, 32, 29, 35, 35, 37, 37, 14, 40, 41, 41, 43, 43, 45, 45, 47, 47, 49, 49, 40, 52, 53, 54, 54, 53, 57, 57, 52, 60, 60, 62, 62, 64, 64, 5, 67, 67, 69, 70, 70, 69, 73, 73], "Labels": ["0", "P08144", "B8Y698", "10.404", "Q2KJQ1", "20.912", "30.882", "P29957", "40.866", "51.000", "Q47R94", "Q1KLC8", "61.000", "Q4A3E0", "70.870", "80.951", "90.967", "101.000", "Q5UZY3", "L8B068", "110.970", "Q8A1G3", "120.830", "D8J7H2", "130.637", "P20845", "141.000", "A5ILB0", "P96107", "150.980", "160.953", "D4P4Y7", "170.999", "T1WDH2", "T1WE96", "181.000", "H9B4I9", "190.759", "B8Y1H0", "A8VWC5", "201.000", "210.842", "Q6WUB6", "220.999", "A0A060DAC6", "230.502", "B1VK33", "240.771", "P06279", "251.000", "P00692", "P06278", "260.866", "271.000", "280.150", "O33476", "O08452", "290.503", "Q2QC8

In [5]:
infer

'{"Comment":"Invalid request: Invalid JSON in command : Infer; JSONObject[\\"Index\\"] not found.","Message":"Request is invalid","Error":4}\n'